# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [30]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url	slides_url
2023-06-13	Water dynamics and sum-frequency generation spectra at electrode/aqueous electrolyte interfaces	Faraday Discussions	The dynamics of water at interfaces between an electrode and an electrolyte is essential for the transport of redox species and for the kinetics of charge transfer reactions next to the electrode. However, while the effects of electrode potential and ion concentration on the electric double layer structure have been extensively studied, a comparable understanding of dynamical aspects is missing. Interfacial water dynamics presents challenges since it is expected to result from the complex combination of water–water, water–electrode and water–ion interactions. Here we perform molecular dynamics simulations of aqueous NaCl solutions at the interface with graphene electrodes, and examine the impact of both ion concentration and electrode potential on interfacial water reorientational dynamics. We 

## Import pandas

We are using the very handy pandas library for dataframes.

In [31]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [32]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,slides_url
0,2023-06-13,Water dynamics and sum-frequency generation sp...,Faraday Discussions,The dynamics of water at interfaces between an...,"Olivieri, J.-F., Hynes, J. T. & Laage, D. Wate...",https://pubs.rsc.org/en/content/articlelanding...,http://jfo.github.io/files/paper1.pdf,http://jfo.github.io/files/slides1.pdf
1,2021-08-24,A Model Electron Transfer Reaction in Confined...,ChemPhysChem,Liquid water confined within nanometer-sized c...,"Olivieri, J.-F., Laage, D. & Hynes, J. T. A Mo...",https://chemistry-europe.onlinelibrary.wiley.c...,http://jfo.github.io/files/paper1.pdf,http://jfo.github.io/files/slides1.pdf
2,2021-04-29,Confined Water’s Dielectric Constant Reduction...,The Journal of Physical Chemistry Letters,Liquid water confined within nanometer-sized c...,"Olivieri, J.-F., Hynes, J. T. & Laage, D. Conf...",https://pubs.acs.org/doi/abs/10.1021/acs.jpcle...,http://jfo.github.io/files/paper1.pdf,http://jfo.github.io/files/slides1.pdf
3,2020-05-21,Setting the photoelectron clock through molecu...,Nature Communications,The interaction of strong laser fields with ma...,"Trabattoni, A. et al. Setting the photoelectro...",https://www.nature.com/articles/s41467-020-162...,http://jfo.github.io/files/paper1.pdf,http://jfo.github.io/files/slides1.pdf
4,2019-08-06,Strong-field photoelectron momentum imaging of...,New Journal of Physics,Photoelectron momentum distributions from stro...,"Wiese, J., Olivieri, J.-F., Trabattoni, A., Tr...",https://iopscience.iop.org/article/10.1088/136...,http://jfo.github.io/files/paper1.pdf,http://jfo.github.io/files/slides1.pdf


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [33]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [ ]:
import os
for row, item in publications.iterrows():
    #md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    md_filename = str(item.pub_date) + ".md"
    #html_filename = str(item.pub_date) + "-" + item.url_slug
    html_filename = str(item.pub_date)
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    if len(str(item.slides_url)) > 5:
        md += "\n[Download slides here](" + item.slides_url + ")\n" 

    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
    print(md_filename)

    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

d3fd00103b.md
cphc.202100351.md
acs.jpclett.1c00447.md
s41467-020-16270-0.md
ab34e8.md


These files are in the publications directory, one directory below where we're working from.

In [35]:
!ls ../_publications/

2009-10-01-paper-title-number-1.md  ab34e8.md
2010-10-01-paper-title-number-2.md  acs.jpclett.1c00447.md
2015-10-01-paper-title-number-3.md  cphc.202100351.md
2024-02-17-paper-title-number-4.md  d3fd00103b.md
2025-06-08-paper-title-number-5.md  s41467-020-16270-0.md


In [36]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

---
title: "Paper Title Number 1"
collection: publications
category: manuscripts
permalink: /publication/2009-10-01-paper-title-number-1
excerpt: 'This paper is about the number 1. The number 2 is left for future work.'
date: 2009-10-01
venue: 'Journal 1'
slidesurl: 'https://academicpages.github.io/files/slides1.pdf'
paperurl: 'https://academicpages.github.io/files/paper1.pdf'
bibtexurl: 'https://academicpages.github.io/files/bibtex1.bib'
citation: 'Your Name, You. (2009). &quot;Paper Title Number 1.&quot; <i>Journal 1</i>. 1(1).'
---
The contents above will be part of a list of publications, if the user clicks the link for the publication than the contents of section will be rendered as a full page, allowing you to provide more information about the paper for the reader. When publications are displayed as a single page, the contents of the above "citation" field will automatically be included below this section in a smaller font.
